In [1]:
!pip install scikit-learn=='1.2.1'

In [20]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import joblib
from sagemaker.sklearn import SKLearnModel
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

In [4]:
df = pd.read_csv("data/original/heart.csv")

In [8]:
X = df.drop("output", axis=1)

In [9]:
y = df["output"]

In [45]:
model = RandomForestClassifier(max_depth=5)

In [47]:
model.fit(X, y)

RandomForestClassifier(max_depth=5)

In [ ]:
model.predict([["1,2,3,3,4"]])

In [37]:
joblib.dump(model, "model.joblib")

['model.joblib']

In [38]:
!tar -czvf model.tar.gz  model.joblib

model.joblib


In [41]:
!aws s3 cp model.tar.gz s3://sagemaker-us-east-1-593538367625/heart_custom_model/

upload: ./model.tar.gz to s3://sagemaker-us-east-1-593538367625/heart_custom_model/model.tar.gz


In [42]:
model = SKLearnModel(
    model_data="s3://sagemaker-us-east-1-593538367625/heart_custom_model/model.tar.gz",
    role="LabRole",
    entry_point="entrypoint.py",
    framework_version="1.2-1")


In [43]:
model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    serielizer=CSVSerializer(),
    deserializer=CSVDeserializer())

------!